# Ingest SEC Data
In this notebook we build a knowledge graph in Neo4j to power AI apps with GraphRAG. The graph integrates asset manager ownership data with company 10-K filings, enabling advanced retrieval and answering complex queries related to asset manager portfolios, companies, and financial topics.

We pull data from two form types, both sourced from the publicly available United States Security & Exchange Commission (SEC) EDGAR database.

__Structured Data on Asset Managers Ownership of Company Equity__: Used to create structured entities and relationships. These are pulled from semi-structured form13 filings. Asset managers with over \$100m AUM are required to submit Form 13 quarterly.  Form 13's are semi-structured XML data.  We parse them into csv files containing the pertinent information on manager ownership using [this code](https://github.com/neo4j-partners/neo4j-sec-edgar-form13).


__Unstructured Text Data__: Used to create embedding vectors for search. Pulled from form10k company sections. These contain quantitative descriptions fo business outlooks for different companies.

## Setup
First, let's install the libraries we're going to need for this lab and the following notebook dependent labs.  We'll also want to reboot the kernel once done.  To do that, go to the "Kernel" menu and click "Restart Kernel and Clear All Outputs."  That will get rid of everything the install statements printed, leaving us with a cleaner notebook to work with.

In [ ]:
%pip install neo4j langchain-aws python-dotenv

Now restart the kernel. That will allow the Python evironment to import the new packages.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
SERVICE_NAME = 'bedrock-runtime'
REGION_NAME = 'us-east-1'

## Establish Neo4j Connection

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

#load neo4j credentials

load_dotenv('cred.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## Load Structured Data from Form 13

In [ ]:
driver.execute_query('CREATE CONSTRAINT unique_company_id IF NOT EXISTS FOR (p:Company) REQUIRE (p.cusip) IS NODE KEY')
driver.execute_query('CREATE CONSTRAINT unique_manager IF NOT EXISTS FOR (p:Manager) REQUIRE (p.managerName) IS NODE KEY')

In [ ]:
driver.execute_query("""
    LOAD CSV WITH HEADERS FROM 'https://neo4j-dataset.s3.amazonaws.com/hands-on-lab/form13-2023-05-11.csv' AS row
    MERGE (c:Company {cusip:row.cusip})
    ON CREATE SET c.companyName=row.companyName;
""")

In [ ]:
driver.execute_query("""
    LOAD CSV WITH HEADERS FROM 'https://neo4j-dataset.s3.amazonaws.com/hands-on-lab/form13-2023-05-11.csv' AS row
    MERGE (m:Manager {managerName:row.managerName});
""")

In [ ]:
with driver.session(database="neo4j") as session:
    result = session.run("""
    LOAD CSV WITH HEADERS FROM "https://neo4j-dataset.s3.amazonaws.com/hands-on-lab/form13-2023-05-11.csv" AS row
    CALL(row) {
        MATCH (m:Manager {managerName:row.managerName})
        MATCH (c:Company {cusip:row.cusip})
        MERGE (m)-[r:OWNS {reportCalendarOrQuarter:date(row.reportCalendarOrQuarter)}]->(c)
        SET r.value = toFloat(row.value), r.shares = toInteger(row.shares)
    } IN TRANSACTIONS OF 1000 ROWS;
    """)
    print(result.consume().counters)

## Load Unstructured Text from Form 10k

To save on tokens & time for purposes of this example, we previously extracted relevant text into zip file using [this code](https://github.com/neo4j-partners/neo4j-sec-edgar/tree/main/form10-k).

### Download and Explore 10k Data

In [ ]:
import boto3

s3 = boto3.resource('s3')
s3.Bucket('neo4j-dataset').download_file('hands-on-lab/form10k.zip', 'form10k.zip')

In [ ]:
!mkdir -p form10k
!unzip -qq -n 'form10k.zip' -d form10k

In [ ]:
import json
with open('form10k/0001830197-22-000038.txt') as f:
    f10_k = json.load(f)

In [ ]:
len(f10_k['item1'])

In [ ]:
f10_k['item1'][:1000]

### Split Text
Test splitting text for more refined embeddings (and to not go over token limits)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = f10_k['item1']

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 15,
    length_function = len,
    is_separator_regex = False,
)
docs = text_splitter.split_text(text)

In [ ]:
print(docs[0])

### Get Bedrock Text Embedding Model

In [ ]:
SERVICE_NAME = 'bedrock-runtime'
REGION_NAME = 'us-east-1'

import boto3
bedrock = boto3.client(
 service_name=SERVICE_NAME,
 region_name=REGION_NAME,
 endpoint_url=f'https://{SERVICE_NAME}.{REGION_NAME}.amazonaws.com'
)

In [ ]:
from langchain_aws import BedrockEmbeddings
embedding_model = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock)

### Generate Embeddings

In [ ]:
# We will need a chunking utility to stay within token limits as we loop through files
def chunks(xs, n=3):
    n = max(1, n)
    return [xs[i:i + n] for i in range(0, len(xs), n)]

In [ ]:
import time

def create_text_embedding_entries(input_text:str, company_name: str, cusip: str):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 2000,
        chunk_overlap  = 15,
        length_function = len,
        is_separator_regex = False,
    )
    docs = text_splitter.split_text(input_text)
    res = []
    seq_id = -1
    for d in chunks(docs):
        embeddings = embedding_model.embed_documents(d)
        
        # throttle so we don't blow through the quota.
        # time.sleep(1)
        
        for i in range(len(d)):
            seq_id += 1
            res.append({'companyName': company_name, 'cusip': cusip, 'seqId': seq_id, 'contextId': company_name + str(seq_id), 'textEmbedding': embeddings[i], 'text': d[i]})
    return res

In [ ]:
import os

file_names = os.listdir('form10k/') #[0:5]
len(file_names)

In [ ]:
%%time

count = 0
embedding_entries = []
for file_name in file_names:
    if '.txt' in file_name:
        count += 1
        if count % 5 == 0:
            print(f'Parsed {count} of {len(file_names)}')
        with open('form10k/' + file_name) as f:
            f10_k = json.load(f)
        embedding_entries.extend(create_text_embedding_entries(f10_k['item1'], f10_k['companyName'], f10_k['cusip']))
len(embedding_entries)

In [ ]:
import pandas as pd
edf = pd.DataFrame(embedding_entries)
edf

### Load Embeddings Into New Document Node Type

In [ ]:
driver.execute_query('CREATE INDEX company_name IF NOT EXISTS FOR (n:Company) ON (n.companyName)')
driver.execute_query('CREATE CONSTRAINT unique_document_id IF NOT EXISTS FOR (n:Document) REQUIRE (n.documentId) IS NODE KEY')

In [ ]:
emb_entries = edf.to_dict(orient='records')

In [ ]:
total = len(emb_entries)
count = 0
for d in chunks(emb_entries, 100):
    driver.execute_query('''
    UNWIND $records AS record
    MATCH(c:Company {cusip:record.cusip})
    MERGE(b:Document {documentId:record.contextId})
    SET b.documentType = 'FORM_10K_ITEM1', b.seqId = record.seqId, b.textEmbedding = record.textEmbedding, b.text = record.text
    MERGE(c)-[:HAS]->(b)
    RETURN count(b) as cnt
    ''', parameters_={'records':d})
    count += len(d)
    print(f'loaded {count} of {total}')

### Analyze Resulting Embeddings

In [ ]:
# Check node count
driver.execute_query('MATCH(doc:Document) RETURN count(doc)').records

In [ ]:
# Check count and percentage of companies with 10-K docs.  Note it is the minority
driver.execute_query('''
MATCH(b:Company)
WITH b, count{(b)-[:HAS]->(d:Document)} AS docCount
WITH count(b) AS total, sum(toInteger(docCount > 0)) AS numWithDocs
RETURN total, numWithDocs, round(100*toFloat(numWithDocs)/toFloat(total), 2) As PercWithDocs
''').records

In [ ]:
# Show duplicates via HAS relationship
driver.execute_query('''
MATCH(b:Company)
RETURN count(b) AS totalCompanies, count(DISTINCT b.companyName) AS uniqueCompanyNames
''').records

In [ ]:
import altair as alt

def generate_chart(df, xcol, ycol, lbl = 'on', color = 'basic', title = '', tooltips = ['documentId'], label = ''):
  chart = alt.Chart(df).mark_circle(size=30).encode(
    x = alt.X(xcol,
        scale=alt.Scale(zero = False),
        axis=alt.Axis(labels = False, ticks = False, domain = False)
    ),
    y = alt.Y(ycol,
        scale=alt.Scale(zero = False),
        axis=alt.Axis(labels = False, ticks = False, domain = False)
    ),
    color= alt.value('#333293') if color == 'basic' else color,
    tooltip=tooltips
    )

  if lbl == 'on':
    text = chart.mark_text(align = 'left', baseline = 'middle', dx = 7, size = 5, color = 'black').encode(text = label, color = alt.value('black'))
  else:
    text = chart.mark_text(align = 'left', baseline = 'middle', dx = 10).encode()

  result = (chart + text).configure(background="#FDF7F0"
        ).properties(
        width = 800,
        height = 500,
        title = title
       ).configure_legend(
  orient = 'bottom', titleFontSize = 18, labelFontSize = 18)
        
  return result

# Reduce dimensionality using PCA
from sklearn.decomposition import PCA

# Function to return the principal components
def get_pc(arr, n):
  pca = PCA(n_components = n)
  embeds_transform = pca.fit_transform(arr)
  return embeds_transform

In [ ]:
res = driver.execute_query("""
    MATCH (c:Company)-[:HAS]->(n:Document) 
    RETURN c.companyName as companyName, n.documentId as documentId, n.text as text, n.textEmbedding as emb LIMIT 1000
""").records
emb_df = pd.DataFrame([i.data() for i in res])
emb_df

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

embeds = np.array(emb_df['emb'].tolist())
embeds_pc2 = get_pc(embeds, 2)

df_clust = pd.concat([emb_df, pd.DataFrame(embeds_pc2)], axis = 1)
n_clusters = 5

kmeans_model = KMeans(n_clusters = n_clusters, n_init = 1, random_state = 0)
classes = kmeans_model.fit_predict(embeds).tolist()
df_clust['cluster'] = (list(map(str,classes)))

df_clust.columns = df_clust.columns.astype(str)
generate_chart(df_clust.iloc[:],'0', '1', lbl = 'off', color = 'cluster', title = 'K-Means Clustering with n Clusters', tooltips = ['documentId', 'text'], label = '')